### Installing packages

In [11]:
%pip install pandas
%pip install numpy
%pip install owlready2
%pip install seaborn
%pip install matplotlib
%pip install cycler
%pip install PyYAML

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.5/167.5 kB 2.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Initializing Imports

In [12]:
# These are boiler plate imports that seem useful
# I should probably delete or comment out the ones that aren't used in this script...

import sys
import os
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
import yaml

import pandas as pd
import numpy as np

import matplotlib as mpl
from cycler import cycler

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# sys.path.append('/Users/rick/HACKING/UTILITIES_IN_HACKING/')
# import utils_general


# this one gives a warning - not fixing for now
from owlready2 import *

# next function is used to generate timestamps that can be used to
# create uniquely named directories and/or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime


print('Initial imports successful')

Initial imports successful


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




Loading SQLite DB

In [13]:
# following https://www.chrisnicoll.net/2020/02/exploring-an-sqlite-database-from-jupyter-notebook/

import sqlite3

dir = '/Users/mihir/'
sqldb = 'wheat-001.db'

# bog-standard read-write connection
conn = sqlite3.connect(dir + sqldb)

# create a cursor object
curs = conn.cursor()

# Every SQLite database has an SQLITE_MASTER table that defines the schema for that database

# Use cursor's fetchall() method to fetch contents from sqlite_master
out1 = curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

print(out1)

out2 = curs.execute("SELECT * FROM sqlite_master WHERE type = 'table'").fetchall()
# print(json.dumps(out2, indent=4))

out3 = curs.execute("SELECT * FROM report").fetchall()
print(json.dumps(out3, indent=4))

conn.close()

[('_InitialConditions',), ('_Messages',), ('Report',), ('_Simulations',), ('_Checkpoints',), ('_Units',), ('_Factors',)]
[
    [
        1,
        1,
        "Field",
        "1900-11-10 12:00:00",
        90.0,
        "HarvestRipe",
        1225.3496679257769,
        17.33616624887376,
        4700.396065766876,
        14.616091528599197,
        0.028970905658757005,
        16224.539616165199,
        470.03960657668756,
        12.031772174766413,
        1436.7843702186733,
        null,
        null
    ],
    [
        1,
        1,
        "Field",
        "1901-11-11 12:00:00",
        90.0,
        "HarvestRipe",
        1300.3608715649557,
        12.620468536831627,
        4691.87215617543,
        10.953580054711585,
        0.02882138902435417,
        16279.132668487218,
        469.18721561754296,
        9.000489889516608,
        1532.3805261997013,
        null,
        null
    ],
    [
        1,
        1,
        "Field",
        "1903-11-04 12:00:00",
     

### Leveraging pandas dataframes with SQLite

In [16]:
# following https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html

import pandas as pd
import sqlite3

dir = '/Users/mihir/'
sqldb = 'wheat-001.db'


# bog-standard read-write connection
conn = sqlite3.connect(dir + sqldb)

# create a cursor object
curs = conn.cursor()


df1 = pd.read_sql_query("SELECT * from sqlite_master WHERE type = 'table'", conn)
print('\nContents of sqlite_master of type table: \n\n')
print(df1.head(30))


df2 = pd.read_sql_query("SELECT * from report", conn)
print('\n\nSome records from the table "report": \n\n')
print(df2.head(30))

conn.close()


Contents of sqlite_master of type table: 


    type                name            tbl_name  rootpage   
0  table  _InitialConditions  _InitialConditions         2  \
1  table           _Messages           _Messages         6   
2  table              Report              Report        92   
3  table        _Simulations        _Simulations        97   
4  table        _Checkpoints        _Checkpoints        99   
5  table              _Units              _Units       101   
6  table            _Factors            _Factors       102   

                                                 sql  
0  CREATE TABLE [_InitialConditions] ("Checkpoint...  
1  CREATE TABLE [_Messages] ("CheckpointID" integ...  
2  CREATE TABLE [Report] ("CheckpointID" integer,...  
3  CREATE TABLE [_Simulations] ("ID" integer,"Nam...  
4  CREATE TABLE [_Checkpoints] ("ID" integer,"Nam...  
5  CREATE TABLE [_Units] ("TableName" text,"Colum...  
6  CREATE TABLE [_Factors] ("CheckpointID" intege...  


Some records fro

### Invoking APSIMX

In [18]:
import subprocess
import os

# note: some of the more recent versions of APSIMX don't work on mac, e.g., the following command
# apsim_cmd = "/Applications/APSIM2023.3.7170.0.app/Contents/Resources/bin/Models"
apsim_cmd = "/Applications/APSIM2023.5.7218.0.app/Contents/Resources/bin/Models"

dir_to_run_in = "/Users/mihir/"
model_name = "wheat-001"

os.chdir(dir_to_run_in)
result1 = subprocess.run(["pwd"], capture_output=True, text=True)
print("result1: \n")
print(result1.stdout)

result2 = subprocess.run([apsim_cmd, model_name + ".apsimx"], capture_output=True, text=True)
print("result2: \n")
print(result2.stdout)

result3 = subprocess.run(["ls -l " + model_name + "*"], shell=True, capture_output=True, text=True)
print("result3: \n ")
print(result3.stdout)

result1: 

/Users/mihir

result2: 


result3: 
 
-rw-------  1 mihir  staff    35533 May 21 23:26 wheat-001.apsimx
-rw-------  1 mihir  staff    31226 May 21 23:18 wheat-001.apsimx.bak
-rw-r--r--  1 mihir  staff  4485120 May 21 23:53 wheat-001.db
-rw-r--r--  1 mihir  staff    32768 May 21 23:26 wheat-001.db-shm
-rw-r--r--  1 mihir  staff  2191872 May 21 23:53 wheat-001.db-wal



### Functions to manipulate APSIMX

In [25]:
def list_model_files(dir, model_name):
    os.chdir(dir)
    arg = 'ls -l ' + model_name + '*'
    result = subprocess.run([arg], shell=True, capture_output=True, text=True)
    return 'The files in ' + dir + ' with prefix "' + model_name + '" are: \n\n' + result.stdout + '\n'

def run_apsimx(dir, model_name):
    apsimx_cmd = "/Applications/APSIM2023.5.7218.0.app/Contents/Resources/bin/Models"
    # note: some of the more recent versions of APSIMX don't work on mac
    # apsim_cmd = "/Applications/APSIM2023.3.7170.0.app/Contents/Resources/bin/Models"
    
    os.chdir(dir)
    try:
        # subprocess.check_output([apsimx_cmd, model_name + '.apsimx'], capture_output=True, text=True) 
        subprocess.check_output([apsimx_cmd, model_name + '.apsimx'], text=True) 
        out = ''
    except subprocess.CalledProcessError as e:
        # following https://stackoverflow.com/questions/24849998/how-to-catch-exception-output-from-python-subprocess-check-output
        out = '\nAn error was found with ' + model_name + '.apsimx, which is as follows: \n'
        out += e.output
        out += '\nEnd of error report\n----------------------------------------------\n'
    return(out)

### Check with soil profile

In [27]:

dir = '/Users/mihir/Desktop/Homework/ECS-289L/soil/'
model_name = "soil-model"

out = run_apsimx(dir, model_name)
if out != '':
    print(out)



An error was found with soil-model.apsimx, which is as follows: 
----------------------------------------------
ERROR in file: /Users/mihir/Desktop/Homework/ECS-289L/soil/soil-model.apsimx
Simulation name: Simulation
Models.Core.SimulationException
 ---> System.Exception: Saturation of 0.410 in layer 5 is above acceptable value of 0.408. You must adjust bulk density to below 1.563 OR saturation to below 0.408
Saturation of 0.415 in layer 6 is above acceptable value of 0.411. You must adjust bulk density to below 1.551 OR saturation to below 0.411

   at Models.Soils.Soil.Check(ISummary summary) in /tmp/ApsimX/Models/Soils/Soil.cs:line 307
   at Models.Soils.Soil.OnDoInitialSummary(Object sender, EventArgs e) in /tmp/ApsimX/Models/Soils/Soil.cs:line 124
   at Models.Clock.OnDoCommence(Object _, CommenceArgs e) in /tmp/ApsimX/Models/Clock.cs:line 304
   at Models.Core.Simulation.Run(CancellationTokenSource cancelToken) in /tmp/ApsimX/Models/Core/Simulation.cs:line 292
   --- End of inne